Example of how to run the LADDIE model and to plot output

In [ ]:
import sys
sys.path.append('../../laddie/src/')
sys.path.append('../src2/')

import xarray as xr
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import cmocean as cmo

from laddie import Laddie
from geometry2 import Geometry
from forcing2 import Forcing

import preprocess as pp

import warnings
warnings.filterwarnings('ignore')
np.seterr(all='ignore')

#%matplotlib widget
#%matplotlib inline
%config InlineBackend.print_figure_kwargs={'bbox_inches':None}
%load_ext autoreload
%autoreload 2

Prepare model

In [ ]:
reg = 'CD'
SMB = 1.0

geom = Geometry(reg)
geom.coarsen(N=2)
geom = geom.create()

#forc = Forcing(geom).sose('R2')
forc = Forcing(geom).tanh(Tdeep=0.5,ztcl=-500)
#forc = Forcing(geom).linear2(S1=34.1,T1=.5)

laddie = Laddie(forc)

In [ ]:
#Interpolate ice velocity onto tmask

ds0 = xr.open_dataset('../data/velocity/ANT_G0240_0000.nc')
ds0 = ds0.reindex(y=list(reversed(ds0.y)))
ds0 = ds0.sel(x=slice(np.min(laddie.x.values),np.max(laddie.x.values)),y=slice(np.min(laddie.y.values),np.max(laddie.y.values)))

VX = 0.*geom.draft.copy()
VY = 0.*geom.draft.copy()

for i,ii in enumerate(laddie.x.values):
    i0 = np.argmin((ii-ds0.x.values)**2)
    for j,jj in enumerate(laddie.y.values):
        j0 = np.argmin((jj-ds0.y.values)**2)
        VX[j,i] = ds0.vx[j0,i0]
        VY[j,i] = ds0.vy[j0,i0]

ds0.close()

VXp1 = (VX+np.roll(VX,-1,axis=1))/2
VYp1 = (VY+np.roll(VY,-1,axis=0))/2
VXm1 = (VX+np.roll(VX,1,axis=1))/2
VYm1 = (VY+np.roll(VY,1,axis=0))/2

In [ ]:
#Bunch of settings

laddie.Ah = 25 #Horizontal viscosity
laddie.Kh = 25 #Horizontal diffusivity

laddie.Cdtop = 1.1e-3 #Top drag coefficient
laddie.minD = 6. #Minimum layer thickness
laddie.dt = 96 #Time step
laddie.nu = 0.6

laddie.slip = .5

laddie.utide = .01

laddie.diagday = .1

laddie.mindrho=None
laddie.correctisf = True
laddie.vcut = 1.

In [ ]:
laddiedays = 1 # number of days to integrate laddie for each ice time step
laddiespinup = 20 # number of days to spin up laddie

laddie.saveday = laddiedays
laddie.restday = laddiedays

ds = laddie.compute(days=laddiespinup)
#ds = laddie.compute(days=1,restartfile=f'{forc.name_forcing}_019')

In [ ]:
laddiedays = 1. # number of days to integrate laddie for each ice time step

laddie.saveday = laddiedays
laddie.restday = laddiedays

dtice = .5 # ice time step in years
Nyears = 50

Ntice = int(Nyears/dtice)

Nbetween = 20

restyear = 65

dsav   = xr.open_dataset(f'../results/CD_1.0_tanh_Tdeep0.5_ztcl-500_{restyear:03.0f}.nc')
laddie.mask = dsav.mask
laddie.zb = dsav.zb
dsav.close()

#laddie.Ah = 20
#laddie.Kh = 20
#laddie.dt = 120

#laddie.dt = 120
#laddie.diagday = .1
#laddie.minD = 6.
#laddie.slip = .5
#laddie.vcut = 1.
#laddie.nu = .8
ds = laddie.compute(days=1,restartfile=f'{forc.name_forcing}_{restyear:03.0f}')

for n in range(Ntice):

    for m in range(Nbetween):
        #Extract old ice thickness
        H = np.where(laddie.tmask==1,-1027/917.*laddie.zb,geom.thickness)

        imask = np.where(laddie.mask>0,1,0)

        #Compute ice convergence
        convN = -(np.maximum(VYp1,0)*H + np.minimum(VYp1,0)*np.roll(H*imask,-1,axis=0))/laddie.dy
        convS = (np.maximum(VYm1,0)*np.roll(H*imask,1,axis=0) + np.minimum(VYm1,0)*H)/laddie.dy
        convE = -(np.maximum(VXp1,0)*H + np.minimum(VXp1,0)*np.roll(H*imask,-1,axis=1))/laddie.dx
        convW = (np.maximum(VXm1,0)*np.roll(H*imask,1,axis=1) + np.minimum(VXm1,0)*H)/laddie.dx
        conv = convN+convS+convE+convW * imask

        #Compute new ice thickness
        melt = laddie.lastmelt

        dH = conv-melt + SMB
        Hnew = H+dH*dtice/Nbetween

        #Recompute ice draft
        zbnew = xr.where(laddie.tmask==1,-917./1027*Hnew,laddie.zb)
        zbnew = xr.where(np.isnan(zbnew),laddie.zb,zbnew)
        laddie.zb = zbnew

        calvmask = np.logical_and(laddie.zb>-30,laddie.tmask*(laddie.ocnxm1+laddie.ocnxp1+laddie.ocnym1+laddie.ocnyp1)>0)
        #calvmask = np.logical_and(laddie.zb>-30,laddie.tmask==1)
        while sum(sum(calvmask))>0:
            print(sum(sum(calvmask)))
            laddie.zb = xr.where(calvmask,0,laddie.zb)
            laddie.mask = xr.where(calvmask,0,laddie.mask)
            pp.create_mask(laddie)
            calvmask = np.logical_and(laddie.zb>-30,laddie.tmask*(laddie.ocnxm1+laddie.ocnxp1+laddie.ocnym1+laddie.ocnyp1)>0)
            #calvmask = np.logical_and(laddie.zb>-30,laddie.tmask==1)

    #Integrate Laddie for next ice time step
    ds = laddie.compute(days=laddiedays,restartfile=laddie.restartfile)